# NETFLIX RECOMMENDATION SYSTEM - CAPSTONE PROJECT

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [70]:
data= pd.read_csv('combined_data_1.txt', header = None, names = ['Cust_id', 'Rating'], usecols = [0,1])

In [71]:
data.head()

,Cust_id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [72]:
data.shape

(24058263, 2)

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24058263 entries, 0 to 24058262
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   Cust_id  object 
 1   Rating   float64
dtypes: float64(1), object(1)
memory usage: 367.1+ MB


In [78]:
# to find the total no of movies

movies_count = len(data[data['Rating'].isnull()])
movies_count

4499

In [80]:
# to find the total no of customers

cust_count = data['Cust_id'].nunique() - movies_count
cust_count

470758

In [82]:
# to find the total ratings
ratings_count = data['Cust_id'].count() - movies_count
ratings_count

24053764

In [84]:
# to find the no of ratings received per rating category

data['Rating'].value_counts().sort_index()

Rating
1.0    1118186
2.0    2439073
3.0    6904181
4.0    8085741
5.0    5506583
Name: count, dtype: int64

In [86]:
# to assign the movie id for each customer's rating

current_movie_id=None
movie_ids=[]
for id in data['Cust_id']:
  if ':' in id:
    current_movie_id=int(id.replace(':',''))
  movie_ids.append(current_movie_id)

data['movie ID']=movie_ids

In [88]:
data

,Cust_id,Rating,movie ID
0,1:,NaN,1
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
...,...,...,...
24058258,2591364,2.0,4499
24058259,1791000,2.0,4499
24058260,512536,5.0,4499
24058261,988963,3.0,4499


In [90]:
data.dropna(inplace=True)

In [92]:
data=data.reset_index().drop('index',axis=1)
data

,Cust_id,Rating,movie ID
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1
...,...,...,...
24053759,2591364,2.0,4499
24053760,1791000,2.0,4499
24053761,512536,5.0,4499
24053762,988963,3.0,4499


In [94]:
# Grouping by 'movie ID' and getting count and mean of ratings
ratings_summary = data.groupby('movie ID')['Rating'].agg(['count', 'mean'])
bench_mark_ratings = round(ratings_summary['count'].quantile(0.70), 0) # Calculating 70th percentilefor movie ratings
print(f"Minimum number of ratings considered per movie - benchmark is {bench_mark_ratings}\n")

#  movies to be dropped
drop_movies_list = ratings_summary[ratings_summary['count'] < bench_mark_ratings].index
print(f"Movie IDs dropped (ratings < benchmark): {list(drop_movies_list)}\n")

data['Cust_id'] = data['Cust_id'].astype(int)

# Grouping by customer and getting count and mean
customer_summary = data.groupby('Cust_id')['Rating'].agg(['count', 'mean'])
cust_benchmark = round(customer_summary['count'].quantile(0.70), 0) # 70th percentile for customer activity
drop_cust_list = customer_summary[customer_summary['count'] < cust_benchmark].index
print(f"Customer IDs dropped (ratings < benchmark): {list(drop_cust_list)}\n")

print(f"Minimum number of reviews required per customer: {cust_benchmark}")

# Dropping under - rated movies and customers
data = data[~data['movie ID'].isin(drop_movies_list)]
data = data[~data['Cust_id'].isin(drop_cust_list)]

print(f"Final shape of the data: {data.shape}")

Minimum number of ratings considered per movie - benchmark is 1799.0

Movie IDs dropped (ratings < benchmark): [1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 19, 20, 21, 22, 23, 24, 25, 27, 29, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 49, 50, 51, 53, 54, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 80, 81, 82, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 105, 106, 107, 109, 112, 113, 114, 115, 116, 117, 119, 120, 121, 123, 124, 125, 126, 128, 129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 142, 144, 145, 146, 147, 149, 150, 151, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 168, 169, 170, 172, 174, 176, 177, 179, 182, 183, 184, 186, 190, 192, 193, 194, 195, 196, 198, 200, 202, 203, 204, 205, 206, 207, 210, 211, 212, 214, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 243, 244, 245, 246, 247, 249, 250, 251, 254, 258, 259, 260, 261, 263, 264, 265, 266, 267, 271, 272, 274, 276, 2

In [96]:
#Create ratings matrix for 'ratings' matrix with Rows = userId, Columns = movieId

df_table = pd.pivot_table(data,values='Rating',index='Cust_id',columns='movie ID')
df_table.shape

(143458, 1350)

In [98]:
df_table

movie ID,3,8,16,17,18,26,28,30,32,33,...,4472,4474,4478,4479,4485,4488,4490,4492,4493,4496
Cust_id,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,5.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,...,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649378,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,3.0,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN


In [100]:
# Read the CSV file with correct column handling
df_title = pd.read_csv(
    'movie_titles.csv',
    encoding="ISO-8859-1",
    header=None,
    usecols=[0, 1, 2],  # Only read the first 3 columns
    names=['Movie_Id', 'Year', 'Name'], 
    on_bad_lines='skip'
)

In [102]:
df_title.set_index('Movie_Id', inplace=True)

In [104]:
df_title.head(15)

,Year,Name
Movie_Id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW
6,1997.0,Sick
7,1992.0,8 Man
8,2004.0,What the #$*! Do We Know!?
9,1991.0,Class of Nuke 'Em High 2


## 1. Find out the list of most popular movies based on user ratings

In [50]:
# Group by movie ID and count number of ratings
movie_popularity = data.groupby('movie ID')['Rating'].count().reset_index()
movie_popularity.rename(columns={'Rating': 'Rating_Count'}, inplace=True)

movie_popularity = movie_popularity.merge(df_title, left_on='movie ID', right_on='Movie_Id', how='left') # Merge with movie titles

# Sort by popularity
most_popular_movies = movie_popularity.sort_values(by='Rating_Count', ascending=False)

# Show top 10 most popular movies
print(f"Top 10 Most Popular Movies (by number of ratings):")
print(most_popular_movies[['Name', 'Rating_Count']].head(10))

Top 10 Most Popular Movies (by number of ratings):
                                                   Name  Rating_Count
587   Pirates of the Caribbean: The Curse of the Bla...        117075
746       Lord of the Rings: The Fellowship of the Ring        102721
1291                                    The Sixth Sense        102376
184                                     American Beauty        101450
1162                                     Bruce Almighty         98545
857                            The Silence of the Lambs         95053
1199                          Finding Nemo (Widescreen)         94235
1331                                    The Italian Job         93886
1191                                            Shrek 2         92893
837                                          Braveheart         91502


## 3.Find which Movies have received the best and worst ratings based on User Rating.

In [54]:
# Step 1: Group ratings by movie and calculate average rating and rating count
movie_ratings = data.groupby('movie ID')['Rating'].agg(['mean', 'count']).reset_index()
movie_ratings.rename(columns={'mean': 'Avg_Rating', 'count': 'Rating_Count'}, inplace=True)

# Step 2: Merge with movie titles to get movie names
movie_ratings = movie_ratings.merge(df_title, left_on='movie ID', right_on='Movie_Id', how='left')

# Optional: Filter to include only movies with a minimum number of ratings (e.g., 50)

filtered_movies = movie_ratings[movie_ratings['Rating_Count'] >= cust_benchmark]

# Step 3: Find best-rated movie(s)
best_movies = filtered_movies[filtered_movies['Avg_Rating'] == filtered_movies['Avg_Rating'].max()]
print("Best Rated Movies are :")
print(best_movies[['Name', 'Avg_Rating', 'Rating_Count']])

# Step 4: Find worst-rated movie(s)
worst_movies = filtered_movies[filtered_movies['Avg_Rating'] == filtered_movies['Avg_Rating'].min()]
print("\n Worst Rated Movies:")
print(worst_movies[['Name', 'Avg_Rating', 'Rating_Count']])

Best Rated Movies are :
                Name  Avg_Rating  Rating_Count
1038  Lost: Season 1    4.665432          4860

 Worst Rated Movies:
                  Name  Avg_Rating  Rating_Count
908  House of the Dead    1.962031          4530


In [56]:
#to install the scikit-surprise library for implementing SVD
!pip install scikit-surprise

In [58]:
import math
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [60]:
# get just top 100K rows for faster computation
reader = Reader() # creating object for the class Reader
surprise_df = Dataset.load_from_df(data[['Cust_id', 'movie ID', 'Rating']][:100000], reader)

# Use the SVD algorithm.
svd = SVD() # creating an object for the class SVD

cross_validate(svd, surprise_df, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9934  0.9959  1.0037  0.9977  0.0044  
MAE (testset)     0.7855  0.7873  0.7978  0.7902  0.0054  
Fit time          2.30    2.50    2.32    2.37    0.09    
Test time         0.50    0.45    0.41    0.45    0.04    


{'test_rmse': array([0.99342935, 0.99588784, 1.00373407]),
 'test_mae': array([0.78551004, 0.78733545, 0.79783054]),
 'fit_time': (2.295365810394287, 2.501833200454712, 2.315941095352173),
 'test_time': (0.5022473335266113, 0.45096921920776367, 0.41007065773010254)}

## 2. Create Model that finds the best suited Movie for one random user

In [63]:
user_712664 = df_title.copy()
user_712664 = user_712664.reset_index()
user_712664 = user_712664[~user_712664['Movie_Id'].isin(drop_movies_list)]

# getting full dataset
surprise_df = Dataset.load_from_df(data[['Cust_id', 'movie ID', 'Rating']], reader)

#create a training set for svd
trainset = surprise_df.build_full_trainset()
svd.fit(trainset)

#Predict the ratings for user_712664
user_712664['Estimate_Score'] = user_712664['Movie_Id'].apply(lambda x: svd.predict(712664, x).est)

user_712664 = user_712664.drop('Movie_Id', axis = 1)
user_712664 = user_712664.sort_values('Estimate_Score', ascending=False)

#Printing the top 10 movie recommendations for user_712664
print(user_712664.head(10))

        Year                            Name  Estimate_Score
871   1954.0                   Seven Samurai        5.000000
3289  1974.0                   The Godfather        5.000000
174   1992.0                  Reservoir Dogs        5.000000
32    2000.0  Aqua Teen Hunger Force: Vol. 1        5.000000
995   1961.0                         Yojimbo        4.976053
3119  1955.0                          Rififi        4.939865
240   1959.0              North by Northwest        4.910369
1031  1992.0                     Hard Boiled        4.903672
3455  2004.0                  Lost: Season 1        4.869163
1594  1949.0                   The Third Man        4.847613
